In [1]:
import torch

import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

from PIL import Image

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")

In [21]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: torch.flatten(x))
])

mnist_trainset = torchvision.datasets.MNIST(root='../datasets/MNIST/', train=True, download=True, transform=transform)
mnist_testset = torchvision.datasets.MNIST(root='../datasets/MNIST', train=False, download=True, transform=transform)

data_loading_params = {'batch_size': 16,
                       'shuffle': True,
                       'num_workers': 6
                      }

train_data = torch.utils.data.DataLoader(mnist_trainset, **data_loading_params)
test_data = torch.utils.data.DataLoader(mnist_testset, **data_loading_params)

In [46]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class NeuralNetwork(nn.Module):
    
    def __init__(self, layer_sizes, activation=F.relu):
        super(NeuralNetwork, self).__init__()
        self.linears = nn.ModuleList(
            [nn.Linear(v, layer_sizes[i+1]) for i, v in enumerate(layer_sizes[:-1])]
        )
        self.activation = activation
        
    def forward(self, x):
        for l in self.linears[:-1]:
            x = self.activation(l(x))

        x = self.linears[-1](x)
            
        return x

In [47]:
def fit(model, train_data, criterion, optimizer, num_epochs=10):

    for epoch in tqdm(range(num_epochs), desc="Epochs", leave=False):
        for local_batch, local_labels in tqdm(train_data, desc="Batches", leave=False):
    
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
                
            optimizer.zero_grad()
            loss = criterion(model(local_batch), local_labels)
    
            loss.backward()
            optimizer.step()

In [48]:
device = "cuda" if torch.cuda.is_available() else "cpu"

LEARNING_RATE = 0.003

model = NeuralNetwork(layer_sizes=[784, 100, 100, 10]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

fit(model, train_data, criterion, optimizer, num_epochs=10)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

In [31]:
def calculate_accuracy(model, data_loader):
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
    
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
                
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
        # Calculate accuracy
    accuracy = 100 * correct / total
    return accuracy

In [49]:
print(calculate_accuracy(model, train_data), calculate_accuracy(model, test_data))

98.23 96.57
